In [ ]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 2.1 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import numpy as np


import matplotlib

Loading data

In [ ]:
from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#Seleção de dados que serão integrados
#Coloque True nos dados que quer que sejam integrados e False nos que não serão integrados

#*************************************************#
#*                                               *#
#*            SELEÇÃO DE DADOS DESEJADOS         *#
#*                                               *#
#*************************************************#

altitude = True                          
clima = True   

solo = True      
classeFungicida = True                       
fungicides = True
era5 = True

plotar = False

In [ ]:
def selectedData():
    global altitude, clima, solo, classeFungicida, fungicides, era5

    selected = ""
    if altitude:
        selected += "+altitude"
    if clima:
        selected += "+climaIBGE"
    if solo:
        selected += "+solo"
    if classeFungicida:
        selected += "+classeFungicida"
    if fungicides:
        selected += "+fungicides"
    if era5:
        selected += "+era5"
    return selected[1:]


selectedData()

'altitude+climaIBGE+solo+classeFungicida+fungicides+era5'

In [ ]:
#DATA_PATH = r"data-2005-2018.xlsx"
DATA_PATH = r"data-2005-2018-Altitude-Clima-Solo-Agrotoxicos-Era5.xlsx"

In [ ]:
def load_main_data():
    return pd.read_excel(io=DATA_PATH, sheet_name="2015-2020")
    
main_data = load_main_data()
main_data.head()

,study,year,location,state,rep,n_spray,brand_name,group,active_ingred,sev,...,Tmax_max_mes_12,Tmax_min_mes_12,Tmin_sd_mes_11,Tmin_min_mes_11,Tmin_max_mes_11,Tmin_mean_mes_11,Tmin_sd_mes_12,Tmin_min_mes_12,Tmin_max_mes_12,Tmin_mean_mes_12
0,252,2015,CAMPO VERDE,MT,1,3,Alto 100,DMI,cyproconazole,71.5,...,1049.9219,941.2969,23.857119,551.6172,664.90625,614.4345,19.531132,588.28906,684.1406,634.0132
1,252,2015,CAMPO VERDE,MT,2,3,Alto 100,DMI,cyproconazole,67.2,...,1049.9219,941.2969,23.857119,551.6172,664.90625,614.4345,19.531132,588.28906,684.1406,634.0132
2,252,2015,CAMPO VERDE,MT,3,3,Alto 100,DMI,cyproconazole,66.0,...,1049.9219,941.2969,23.857119,551.6172,664.90625,614.4345,19.531132,588.28906,684.1406,634.0132
3,252,2015,CAMPO VERDE,MT,4,3,Alto 100,DMI,cyproconazole,64.0,...,1049.9219,941.2969,23.857119,551.6172,664.90625,614.4345,19.531132,588.28906,684.1406,634.0132
4,252,2015,CAMPO VERDE,MT,1,3,Aproach Prima,QOI_DMI,picoxystrobin + cyproconazole,42.1,...,1049.9219,941.2969,23.857119,551.6172,664.90625,614.4345,19.531132,588.28906,684.1406,634.0132


In [ ]:
#main_data.to_excel("teste.xlsx")

In [ ]:
def load_location_data():
    return pd.read_excel(io=DATA_PATH, sheet_name="map")
location_data = load_location_data()
location_data

,location,state,long,lat,altitude,desc_completa,distribuicao_umidade,tipo_umidade,legenda,ordem,subordem,grande_grupo,subgrupos,textura,relevo
0,LUÍS EDUARDO MAGALHÃES,BA,-45.780000,-12.0910,768.531308,"Tropical Brasil Central, quente - média > 18° ...",4 a 5 meses secos,semi-úmido,LVAd - Latossolo Vermelho-Amarelo Distrófico,LATOSSOLO,VERMELHO-AMARELO,Distrófico,típico,média,plano
1,PLANALTINA,DF,-47.652000,-15.6220,1037.706282,"Tropical Brasil Central, subquente - média ent...",4 a 5 meses secos,semi-úmido,Área Urbana,NaN,NaN,NaN,NaN,NaN,NaN
2,ANÁPOLIS,GO,-48.953000,-16.3200,1023.435239,"Tropical Brasil Central, subquente - média ent...",4 a 5 meses secos,semi-úmido,Área Urbana,NaN,NaN,NaN,NaN,NaN,NaN
3,GOIÂNIA,GO,-49.265000,-16.6870,753.922667,"Tropical Brasil Central, quente - média > 18° ...",4 a 5 meses secos,semi-úmido,Área Urbana,NaN,NaN,NaN,NaN,NaN,NaN
4,JATAÍ,GO,-51.721000,-17.8750,724.000664,"Tropical Brasil Central, quente - média > 18° ...",3 meses secos,úmido,Área Urbana,NaN,NaN,NaN,NaN,NaN,NaN
5,RIO VERDE,GO,-50.920000,-17.7930,743.307482,"Tropical Brasil Central, subquente - média ent...",3 meses secos,úmido,Área Urbana,NaN,NaN,NaN,NaN,NaN,NaN
6,SANTO ANTONIO DE GOIAS,GO,-49.308000,-16.4850,821.000000,"Tropical Brasil Central, quente - média > 18° ...",4 a 5 meses secos,semi-úmido,LVw - Latossolo Vermelho Ácrico,LATOSSOLO,VERMELHO,Ácrico,típico,argilosa,suave \nondulado e ondulado
7,SÃO MIGUEL DO PASSA QUATRO,GO,-48.750000,-16.8600,853.699984,"Tropical Brasil Central, subquente - média ent...",4 a 5 meses secos,semi-úmido,LVw - Latossolo Vermelho Ácrico,LATOSSOLO,VERMELHO,Ácrico,típico,argilosa,suave \nondulado
8,SENADOR CANEDO,GO,-49.092000,-16.7090,783.822735,"Tropical Brasil Central, quente - média > 18° ...",4 a 5 meses secos,semi-úmido,Área Urbana,NaN,NaN,NaN,NaN,NaN,NaN
9,SILVÂNIA,GO,-48.340000,-16.4700,920.078087,"Tropical Brasil Central, subquente - média ent...",4 a 5 meses secos,semi-úmido,CXbd - Cambissolo Háplico Tb Distrófico,CAMBISSOLO,HÁPLICO,Tb Distrófico,típico e léptico,média cascalhenta,ondulado e forte \nondulado


In [ ]:
#Carrega as informações dos fungicidas do dataset que estão na aba 'fungicides' do excel

fungicides_data = ""

def load_fungicides():
    return pd.read_excel(io=DATA_PATH, sheet_name="fungicides")

if (fungicides):
  fungicides_data = load_fungicides()
  fungicides_data

Características climaticas: ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#Faz a conversão de texto para numero do tipo de umidade
valores = []

if(clima):
  for index, row in location_data.iterrows():
    if row["tipo_umidade"] == "semi-úmido":
      valores.append(0)
      continue
    if row["tipo_umidade"] == "úmido":
      valores.append(1)
      continue
    if row["tipo_umidade"] == "super-úmido":
      valores.append(2)
      continue

if(clima):
  print(valores)
  location_data.insert(5, 'tipo_umidade_EN', valores)
#location_data.drop('tipo_umidade', axis=1, inplace=True)
#location_data

[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 0]


In [ ]:
#Faz a conversão de texto para numero da distribuição de umidade
valores = []

if(clima):
  for index, row in location_data.iterrows():
    if row["distribuicao_umidade"] == "sem seca":
      valores.append(0)
      continue
    if row["distribuicao_umidade"] == "subseca":
      valores.append(1)
      continue
    if row["distribuicao_umidade"] == "1 a 2 meses secos":
      valores.append(2)
      continue
    if row["distribuicao_umidade"] == "3 meses secos":
      valores.append(3)
      continue
    if row["distribuicao_umidade"] == "4 a 5 meses secos":
      valores.append(4)
      continue

if(clima):
  print(valores)
  location_data.insert(6, 'distribuicao_umidade_EN', valores)
#location_data.drop('distribuicao_umidade', axis=1, inplace=True)
#location_data

[4, 4, 4, 4, 3, 3, 4, 4, 4, 4, 4, 2, 2, 2, 2, 3, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 0, 0, 1, 4, 0, 1, 2, 0, 0, 0, 0, 0, 0, 2, 4]


In [ ]:
#Faz a conversão de texto para numero da descrição completa das informaçõeos climáticas
valores1 = []
valores2 = []

if(clima):
  for index, row in location_data.iterrows():
    if row["desc_completa"].find("Tropical Brasil Central") != -1:
      valores1.append(0)
      continue
    if row["desc_completa"].find("Equatorial") != -1:
      valores1.append(1)
      continue
    if row["desc_completa"].find("Temperado") != -1:
      valores1.append(2)
      continue
    print(row["desc_completa"], index)

  for index, row in location_data.iterrows():
    if row["desc_completa"].find("quente - média > 18° C em todos os meses") != -1:
      valores2.append(0)
      continue
    if row["desc_completa"].find("subquente - média entre 15 e 18 ° C em pelo menos 1 mês") != -1:
      valores2.append(1)
      continue
    if row["desc_completa"].find("mesotérmico brando - média entre 10 e 15° C") != -1:
      valores2.append(2)
      continue
  
  

if(clima):
  print(valores1)
  print(valores2)
  location_data.insert(7, 'desc_completa_EN_P1', valores1)
  location_data.insert(8, 'desc_completa_EN_P2', valores2)
#location_data.drop('desc_completa', axis=1, inplace=True)
#location_data

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0]
[0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 0, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 0]


In [ ]:
#Faz one hot encoding da descrição completa das informações climáticas
def convert_desc_completa(data=location_data):
    data["OH_desc_completa_Tropical"] = (data["desc_completa"].str.contains(pat = 'Tropical Brasil Central')).astype(int)
    data["OH_desc_completa_Equatorial"] = (data["desc_completa"].str.contains(pat = 'Equatorial')).astype(int)
    data["OH_desc_completa_Temperado"] = (data["desc_completa"].str.contains(pat = 'Temperado')).astype(int)
    
    data["OH_desc_completa_Quente"] = (data["desc_completa"].str.contains(pat = 'quente - média > 18° C em todos os meses')).astype(int)
    data["OH_desc_completa_Subquente"] = (data["desc_completa"].str.contains(pat = 'subquente - média entre 15 e 18 ° C em pelo menos 1 mês')).astype(int)
    data["OH_desc_completa_Mesotermico"] = (data["desc_completa"].str.contains(pat = 'mesotérmico brando - média entre 10 e 15° C')).astype(int)
    #data["OH_desc_completa_Subquente"] = (data["desc_completa"].str.contains(pat = 'subquente - média entre 15 e 18º C em pelo menos 1 mês')).astype(int)

if(clima):
  convert_desc_completa()

In [ ]:
location_data

,location,state,long,lat,altitude,tipo_umidade_EN,distribuicao_umidade_EN,desc_completa_EN_P1,desc_completa_EN_P2,desc_completa,...,grande_grupo,subgrupos,textura,relevo,OH_desc_completa_Tropical,OH_desc_completa_Equatorial,OH_desc_completa_Temperado,OH_desc_completa_Quente,OH_desc_completa_Subquente,OH_desc_completa_Mesotermico
0,LUÍS EDUARDO MAGALHÃES,BA,-45.780000,-12.0910,768.531308,0,4,0,0,"Tropical Brasil Central, quente - média > 18° ...",...,Distrófico,típico,média,plano,1,0,0,1,0,0
1,PLANALTINA,DF,-47.652000,-15.6220,1037.706282,0,4,0,1,"Tropical Brasil Central, subquente - média ent...",...,NaN,NaN,NaN,NaN,1,0,0,0,1,0
2,ANÁPOLIS,GO,-48.953000,-16.3200,1023.435239,0,4,0,1,"Tropical Brasil Central, subquente - média ent...",...,NaN,NaN,NaN,NaN,1,0,0,0,1,0
3,GOIÂNIA,GO,-49.265000,-16.6870,753.922667,0,4,0,0,"Tropical Brasil Central, quente - média > 18° ...",...,NaN,NaN,NaN,NaN,1,0,0,1,0,0
4,JATAÍ,GO,-51.721000,-17.8750,724.000664,1,3,0,0,"Tropical Brasil Central, quente - média > 18° ...",...,NaN,NaN,NaN,NaN,1,0,0,1,0,0
5,RIO VERDE,GO,-50.920000,-17.7930,743.307482,1,3,0,1,"Tropical Brasil Central, subquente - média ent...",...,NaN,NaN,NaN,NaN,1,0,0,0,1,0
6,SANTO ANTONIO DE GOIAS,GO,-49.308000,-16.4850,821.000000,0,4,0,0,"Tropical Brasil Central, quente - média > 18° ...",...,Ácrico,típico,argilosa,suave \nondulado e ondulado,1,0,0,1,0,0
7,SÃO MIGUEL DO PASSA QUATRO,GO,-48.750000,-16.8600,853.699984,0,4,0,1,"Tropical Brasil Central, subquente - média ent...",...,Ácrico,típico,argilosa,suave \nondulado,1,0,0,0,1,0
8,SENADOR CANEDO,GO,-49.092000,-16.7090,783.822735,0,4,0,0,"Tropical Brasil Central, quente - média > 18° ...",...,NaN,NaN,NaN,NaN,1,0,0,1,0,0
9,SILVÂNIA,GO,-48.340000,-16.4700,920.078087,0,4,0,1,"Tropical Brasil Central, subquente - média ent...",...,Tb Distrófico,típico e léptico,média cascalhenta,ondulado e forte \nondulado,1,0,0,0,1,0


Características do solo: ------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#Faz a conversão de texto para numero da legenda do tipo de solo, precisa usar o dataset 'data-2005-2018-Solo'
valores = []

if(solo):
  for index, row in location_data.iterrows():
    if row["legenda"] == "LVAd - Latossolo Vermelho-Amarelo Distrófico":
      valores.append(0)
      continue
    if row["legenda"] == "LVw - Latossolo Vermelho Ácrico":
      valores.append(1)
      continue
    if row["legenda"] == "CXbd - Cambissolo Háplico Tb Distrófico":
      valores.append(2)
      continue
    if row["legenda"] == "LVd - Latossolo Vermelho Distrófico":
      valores.append(3)
      continue
    if row["legenda"] == "PVAe - Argissolo Vermelho-Amarelo Eutrófico":
      valores.append(4)
      continue
    if row["legenda"] == "LVaf - Latossolo Vermelho Aluminoférrico":
      valores.append(5)
      continue
    if row["legenda"] == "LVwf - Latossolo Vermelho Acriférrico":
      valores.append(6)
      continue
    if row["legenda"] == "NVe - Nitossolo Vermelho Eutrófico":
      valores.append(7)
      continue
    if row["legenda"] == "Área Urbana":
      valores.append(np.nan)
      continue
    if row["legenda"] == "Corpo d'água continental":
      valores.append(np.nan)
      continue

if(solo):
  print(valores)
  location_data.insert(8, 'legenda_EN', valores)
#location_data.drop('desc_completa', axis=1, inplace=True)
#location_data

[0, nan, nan, nan, nan, nan, 1, 1, nan, 2, nan, 3, 7, 3, 3, 1, 3, 3, 3, 3, 3, nan, 2, 0, 0, nan, 4, 3, 0, 3, 0, nan, nan, nan, 3, nan, nan, 3, nan, nan, 5, nan, nan, nan, 6, nan]


In [ ]:
#Faz a conversão de texto para numero da ordem do solo
valores = []

if(solo):
  for index, row in location_data.iterrows():
    if row["ordem"] == "LATOSSOLO":
      valores.append(0)
      continue
    if row["ordem"] == "CAMBISSOLO":
      valores.append(1)
      continue
    if row["ordem"] == "NITOSSOLO":
      valores.append(2)
      continue
    if row["ordem"] == "ARGISSOLO":
      valores.append(3)
      continue
    
    valores.append(np.nan)

if(solo):
  print(valores)
  location_data.insert(9, 'ordem_EN', valores)
#location_data.drop('desc_completa', axis=1, inplace=True)
#location_data

[0, nan, nan, nan, nan, nan, 0, 0, nan, 1, nan, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, nan, 1, 0, 0, nan, 3, 0, 0, 0, 0, nan, nan, nan, 0, nan, nan, 0, nan, nan, 0, nan, nan, nan, 0, nan]


In [ ]:
#Faz a conversão de texto para numero da subordem do solo
valores = []

if(solo):
  for index, row in location_data.iterrows():
    if row["subordem"] == "Distrófico":
      valores.append(0)
      continue
    if row["subordem"] == "HÁPLICO":
      valores.append(1)
      continue
    if row["subordem"] == "VERMELHO":
      valores.append(2)
      continue
    if row["subordem"] == "VERMELHO-AMARELO":
      valores.append(3)
      continue

    valores.append(np.nan)
 
if(solo):
  print(valores)
  location_data.insert(10, 'subordem_EN', valores)
#location_data.drop('desc_completa', axis=1, inplace=True)
#location_data

[3, nan, nan, nan, nan, nan, 2, 2, nan, 1, nan, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, nan, 1, 3, 3, nan, 3, 2, 3, 2, 3, nan, nan, nan, 2, nan, nan, 2, nan, nan, 2, nan, nan, nan, 2, nan]


In [ ]:
#Faz a conversão de texto para numero do grande grupo do solo
valores = []

if(solo):
  for index, row in location_data.iterrows():
    if row["grande_grupo"] == "Distrófico":
      valores.append(0)
      continue
    if row["grande_grupo"] == "Ácrico":
      valores.append(1)
      continue
    if row["grande_grupo"] == "Tb Distrófico":
      valores.append(2)
      continue
    if row["grande_grupo"] == "Eutrófico":
      valores.append(3)
      continue
    if row["grande_grupo"] == "Aluminoférrico":
      valores.append(4)
      continue
    if row["grande_grupo"] == "Acriférrico":
      valores.append(5)
      continue
    valores.append(np.nan)

if(solo):
  print(valores)
  location_data.insert(11, 'grande_grupo_EN', valores)
#location_data.drop('desc_completa', axis=1, inplace=True)
#location_data

[0, nan, nan, nan, nan, nan, 1, 1, nan, 2, nan, 0, 3, 0, 0, 1, 0, 0, 0, 0, 0, nan, 2, 0, 0, nan, 3, 0, 0, 0, 0, nan, nan, nan, 0, nan, nan, 0, nan, nan, 4, nan, nan, nan, 5, nan]


In [ ]:
#Faz a conversão de texto para numero do subgrupo do solo
valores = []

if(solo):
  for index, row in location_data.iterrows():
    if row["subgrupos"] == "típico":
      valores.append(0)
      continue
    if row["subgrupos"] == "típico e léptico":
      valores.append(1)
      continue
    if row["subgrupos"] == "chernossólico, plintossólico":
      valores.append(2)
      continue
    if row["subgrupos"] == "latossólico":
      valores.append(3)
      continue
    valores.append(np.nan)

if(solo):
  print(valores)
  location_data.insert(12, 'subgrupos_EN', valores)
#location_data.drop('desc_completa', axis=1, inplace=True)
#location_data

[0, nan, nan, nan, nan, nan, 0, 0, nan, 1, nan, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, nan, 0, 0, 0, nan, 2, 0, 0, 0, 0, nan, nan, nan, 0, nan, nan, 0, nan, nan, 0, nan, nan, nan, 0, nan]


In [ ]:
#Faz a conversão de texto para numero da textura do solo
valores = []

if(solo):
  for index, row in location_data.iterrows():
    if row["textura"] == "média":
      valores.append(0)
      continue
    if row["textura"] == "argilosa":
      valores.append(1)
      continue
    if row["textura"] == "média cascalhenta":
      valores.append(2)
      continue
    if row["textura"] == "muito argilosa":
      valores.append(3)
      continue
    if row["textura"] == "muito argilosa e argilosa":
      valores.append(4)
      continue
    if row["textura"] == "argilosa e muito argilosa":
      valores.append(5)
      continue
    if row["textura"] == "média e argilosa":
      valores.append(6)
      continue
    if row["textura"] == "média/argilosa":
      valores.append(7)
      continue
    
    valores.append(np.nan)

if(solo):
  print(valores)
  location_data.insert(13, 'textura_EN', valores)
#location_data.drop('desc_completa', axis=1, inplace=True)
#location_data

[0, nan, nan, nan, nan, nan, 1, 1, nan, 2, nan, 0, 3, 0, 4, 4, 0, 0, 5, 5, 0, nan, 6, 1, 0, nan, 7, 1, 1, 1, 1, nan, nan, nan, 0, nan, nan, 3, nan, nan, 3, nan, nan, nan, 5, nan]


In [ ]:
#Faz a conversão de texto para numero do relevo
valores = []

if(solo):
  for index, row in location_data.iterrows():
    if row["relevo"] == "plano":
      valores.append(0)
      continue
    if row["relevo"] == "suave \\nondulado e ondulado":
      valores.append(1)
      continue
    if row["relevo"] == "suave \\nondulado":
      valores.append(2)
      continue
    if row["relevo"] == "ondulado e forte \\nondulado":
      valores.append(3)
      continue
    if row["relevo"] == "plano e suave \\nondulado":
      valores.append(4)
      continue
    if row["relevo"] == "suave \\nondulado e plano":
      valores.append(5)
      continue
    if row["relevo"] == "ondulado e suave \\nondulado":
      valores.append(6)
      continue
    
    valores.append(np.nan)

if(solo):
  print(valores)
  location_data.insert(14, 'relevo_EN', valores)
#location_data.drop('desc_completa', axis=1, inplace=True)
#location_data

[0, nan, nan, nan, nan, nan, 1, 2, nan, 3, nan, 4, 4, 4, 4, 0, 4, 4, 0, 0, 4, nan, 1, 0, 5, nan, 6, 4, 0, 4, 4, nan, nan, nan, 4, nan, nan, 2, nan, nan, 1, nan, nan, nan, 2, nan]


Dados dos agrotóxicos: ------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#Converte cada brand name para um número e coloca como uma nova coluna
valores = []
nova_coluna = []

for index, row in main_data.iterrows():
  if str(row["brand_name"]).strip() in valores:
     nova_coluna.append(valores.index(str(row["brand_name"]).strip()))
  else:
     valores.append(str(row["brand_name"]).strip())
     nova_coluna.append(valores.index(str(row["brand_name"]).strip()))

if(fungicides):
  main_data.insert(11, 'brand_name_EN', nova_coluna)

In [ ]:
def convert_city_to_location(main_data=main_data,location_data=location_data):
    main_data = main_data.merge(right=location_data, how='outer', on=['location', 'state'])
    return main_data
            
main_data = convert_city_to_location()
main_data

,study,year,location,state,rep,n_spray,brand_name,group,active_ingred,sev,...,grande_grupo,subgrupos,textura,relevo,OH_desc_completa_Tropical,OH_desc_completa_Equatorial,OH_desc_completa_Temperado,OH_desc_completa_Quente,OH_desc_completa_Subquente,OH_desc_completa_Mesotermico
0,252,2015,CAMPO VERDE,MT,1,3,Alto 100,DMI,cyproconazole,71.5,...,Distrófico,típico,média,plano e suave \nondulado,1,0,0,1,0,0
1,252,2015,CAMPO VERDE,MT,2,3,Alto 100,DMI,cyproconazole,67.2,...,Distrófico,típico,média,plano e suave \nondulado,1,0,0,1,0,0
2,252,2015,CAMPO VERDE,MT,3,3,Alto 100,DMI,cyproconazole,66.0,...,Distrófico,típico,média,plano e suave \nondulado,1,0,0,1,0,0
3,252,2015,CAMPO VERDE,MT,4,3,Alto 100,DMI,cyproconazole,64.0,...,Distrófico,típico,média,plano e suave \nondulado,1,0,0,1,0,0
4,252,2015,CAMPO VERDE,MT,1,3,Aproach Prima,QOI_DMI,picoxystrobin + cyproconazole,42.1,...,Distrófico,típico,média,plano e suave \nondulado,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12501,417,2020,QUERÊNCIA,MT,4,4,Programa 1,NaN,NaN,30.0,...,Distrófico,típico,argilosa,plano,1,0,0,1,0,0
12502,417,2020,QUERÊNCIA,MT,1,4,Programa 2,NaN,NaN,28.0,...,Distrófico,típico,argilosa,plano,1,0,0,1,0,0
12503,417,2020,QUERÊNCIA,MT,2,4,Programa 2,NaN,NaN,30.0,...,Distrófico,típico,argilosa,plano,1,0,0,1,0,0
12504,417,2020,QUERÊNCIA,MT,3,4,Programa 2,NaN,NaN,32.0,...,Distrófico,típico,argilosa,plano,1,0,0,1,0,0


In [ ]:
#Função utilizada para remover as linhas que tem NaN na coluna de 'active_ingredients'
def remove_active_NaN(data=main_data):
    data.dropna(subset=['active_ingred'],inplace=True)

#Corrige os nomes dos ingredientes ativos para estarem igual a aba 'fungicides' e poder ser feito o join
def convert_active_ingred_group(data=main_data):
    #remove_active_NaN(data)
    data["active_ingred"] = (data["active_ingred"].str.lower())
    data["active_ingred"] = (data["active_ingred"].str.replace(" ", ""))

if(fungicides):
  convert_active_ingred_group()
main_data 

,study,year,location,state,rep,n_spray,brand_name,group,active_ingred,sev,...,grande_grupo,subgrupos,textura,relevo,OH_desc_completa_Tropical,OH_desc_completa_Equatorial,OH_desc_completa_Temperado,OH_desc_completa_Quente,OH_desc_completa_Subquente,OH_desc_completa_Mesotermico
0,252,2015,CAMPO VERDE,MT,1,3,Alto 100,DMI,cyproconazole,71.5,...,Distrófico,típico,média,plano e suave \nondulado,1,0,0,1,0,0
1,252,2015,CAMPO VERDE,MT,2,3,Alto 100,DMI,cyproconazole,67.2,...,Distrófico,típico,média,plano e suave \nondulado,1,0,0,1,0,0
2,252,2015,CAMPO VERDE,MT,3,3,Alto 100,DMI,cyproconazole,66.0,...,Distrófico,típico,média,plano e suave \nondulado,1,0,0,1,0,0
3,252,2015,CAMPO VERDE,MT,4,3,Alto 100,DMI,cyproconazole,64.0,...,Distrófico,típico,média,plano e suave \nondulado,1,0,0,1,0,0
4,252,2015,CAMPO VERDE,MT,1,3,Aproach Prima,QOI_DMI,picoxystrobin+cyproconazole,42.1,...,Distrófico,típico,média,plano e suave \nondulado,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12501,417,2020,QUERÊNCIA,MT,4,4,Programa 1,NaN,NaN,30.0,...,Distrófico,típico,argilosa,plano,1,0,0,1,0,0
12502,417,2020,QUERÊNCIA,MT,1,4,Programa 2,NaN,NaN,28.0,...,Distrófico,típico,argilosa,plano,1,0,0,1,0,0
12503,417,2020,QUERÊNCIA,MT,2,4,Programa 2,NaN,NaN,30.0,...,Distrófico,típico,argilosa,plano,1,0,0,1,0,0
12504,417,2020,QUERÊNCIA,MT,3,4,Programa 2,NaN,NaN,32.0,...,Distrófico,típico,argilosa,plano,1,0,0,1,0,0


In [ ]:
#realiza o join das tabelas de dados com a de fungicidas
def join_fungicide_data(main_data=main_data, fungicides_data=fungicides_data):
    main_data = main_data.merge(right=fungicides_data, how='outer')
    return main_data

if(fungicides):           
  main_data = join_fungicide_data()
  main_data

In [ ]:
main_data.columns

Index(['study', 'year', 'location', 'state', 'rep', 'n_spray', 'brand_name',
       'group', 'active_ingred', 'sev',
       ...
       'OH_CYPR', 'OH_PICO', 'OH_TRIF', 'OH_PROT', 'OH_TEBU', 'OH_PYRA',
       'OH_FLUX', 'OH_BENZ', 'OH_EPOX', 'OH_BIXA'],
      dtype='object', length=130)

In [ ]:
#main_data.to_excel("avaliacao.xlsx")

Treating data

In [ ]:
#Correção necessário pelo motivo de alguma anomalia causada quando une a tabela
  # de map com a tabela principal quando tem os dados do era5
#if(era5 and altitude):
#  main_data.drop(index=12506, inplace=True)
  #main_data

In [ ]:
#ef reduce_year(main_data=main_data):
#    main_data["year"] -= 2005
#    return main_data
#main_data = reduce_year()
#main_data

In [ ]:
def remove_active_NaN(data=main_data):
    data.dropna(subset=['group'],inplace=True)
    #data.dropna(subset=['group'],inplace=True)

def convert_active_ingred_group(data=main_data):
    remove_active_NaN(data)
    data["CHECK"] = (data["group"].str.contains(pat = 'CHECK')).astype(int)
    data["DMI"] = (data["group"].str.contains(pat = 'DMI')).astype(int)
    # data["INS"] = (data["group"].str.contains(pat = 'INS')).astype(int)
    data["MBC"] = (data["group"].str.contains(pat = 'MBC')).astype(int)
    data["SDHI"] = (data["group"].str.contains(pat = 'SDHI')).astype(int)
    data["QOI"] = (data["group"].str.contains(pat = 'QOI')).astype(int)
    data["COP"] = (data["group"].str.contains(pat = 'COP')).astype(int)
    data["DIT"] = (data["group"].str.contains(pat = 'DIT')).astype(int)

if(classeFungicida):
  convert_active_ingred_group()
  main_data    

In [ ]:
def convert_sev_to_float(data = main_data):
    # data["sev"] = data["sev"].astype(float)
    data["sev"] = pd.to_numeric(arg=data["sev"], errors="coerce")
    # data["sev"].dropna(inplace=True)
    data["sev"] = data["sev"].fillna(0)
convert_sev_to_float()

In [ ]:
main_data

,study,year,location,state,rep,n_spray,brand_name,group,active_ingred,sev,...,OH_BENZ,OH_EPOX,OH_BIXA,CHECK,DMI,MBC,SDHI,QOI,COP,DIT
0,252,2015,CAMPO VERDE,MT,1,3,Alto 100,DMI,cyproconazole,71.5,...,0.0,0.0,0.0,0,1,0,0,0,0,0
1,252,2015,CAMPO VERDE,MT,2,3,Alto 100,DMI,cyproconazole,67.2,...,0.0,0.0,0.0,0,1,0,0,0,0,0
2,252,2015,CAMPO VERDE,MT,3,3,Alto 100,DMI,cyproconazole,66.0,...,0.0,0.0,0.0,0,1,0,0,0,0,0
3,252,2015,CAMPO VERDE,MT,4,3,Alto 100,DMI,cyproconazole,64.0,...,0.0,0.0,0.0,0,1,0,0,0,0,0
4,269,2015,CAMPO VERDE,MT,1,3,Alto 100,DMI,cyproconazole,80.0,...,0.0,0.0,0.0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12501,421,2020,SÃO MIGUEL DO PASSA QUATRO,GO,4,4,Audaz/ Aumenax,SDHI_COP,fluxapyroxad+copperoxychloride,9.1,...,NaN,NaN,NaN,0,0,0,1,0,1,0
12502,417,2020,QUERÊNCIA,MT,1,4,Audaz/ Aumenax,SDHI_COP,fluxapyroxad+copperoxychloride,48.0,...,NaN,NaN,NaN,0,0,0,1,0,1,0
12503,417,2020,QUERÊNCIA,MT,2,4,Audaz/ Aumenax,SDHI_COP,fluxapyroxad+copperoxychloride,48.0,...,NaN,NaN,NaN,0,0,0,1,0,1,0
12504,417,2020,QUERÊNCIA,MT,3,4,Audaz/ Aumenax,SDHI_COP,fluxapyroxad+copperoxychloride,46.0,...,NaN,NaN,NaN,0,0,0,1,0,1,0


In [ ]:
main_dataTmp = main_data.copy()

In [ ]:
main_dataTmp

,study,year,location,state,rep,n_spray,brand_name,group,active_ingred,sev,...,OH_BENZ,OH_EPOX,OH_BIXA,CHECK,DMI,MBC,SDHI,QOI,COP,DIT
0,252,2015,CAMPO VERDE,MT,1,3,Alto 100,DMI,cyproconazole,71.5,...,0.0,0.0,0.0,0,1,0,0,0,0,0
1,252,2015,CAMPO VERDE,MT,2,3,Alto 100,DMI,cyproconazole,67.2,...,0.0,0.0,0.0,0,1,0,0,0,0,0
2,252,2015,CAMPO VERDE,MT,3,3,Alto 100,DMI,cyproconazole,66.0,...,0.0,0.0,0.0,0,1,0,0,0,0,0
3,252,2015,CAMPO VERDE,MT,4,3,Alto 100,DMI,cyproconazole,64.0,...,0.0,0.0,0.0,0,1,0,0,0,0,0
4,269,2015,CAMPO VERDE,MT,1,3,Alto 100,DMI,cyproconazole,80.0,...,0.0,0.0,0.0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12501,421,2020,SÃO MIGUEL DO PASSA QUATRO,GO,4,4,Audaz/ Aumenax,SDHI_COP,fluxapyroxad+copperoxychloride,9.1,...,NaN,NaN,NaN,0,0,0,1,0,1,0
12502,417,2020,QUERÊNCIA,MT,1,4,Audaz/ Aumenax,SDHI_COP,fluxapyroxad+copperoxychloride,48.0,...,NaN,NaN,NaN,0,0,0,1,0,1,0
12503,417,2020,QUERÊNCIA,MT,2,4,Audaz/ Aumenax,SDHI_COP,fluxapyroxad+copperoxychloride,48.0,...,NaN,NaN,NaN,0,0,0,1,0,1,0
12504,417,2020,QUERÊNCIA,MT,3,4,Audaz/ Aumenax,SDHI_COP,fluxapyroxad+copperoxychloride,46.0,...,NaN,NaN,NaN,0,0,0,1,0,1,0


Normalização de valores e remoção de NaN ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
if(era5): #perde 1200 linhas
   main_dataTmp = main_dataTmp[main_dataTmp['year'] != 2020.0]

In [ ]:
from sklearn.preprocessing import StandardScaler

# Cria uma instância da classe StandardScaler
scalerStandart = StandardScaler()

# Normaliza a coluna desejada
main_dataTmp['lat_nrmlzd_Standart'] = scalerStandart.fit_transform(main_dataTmp[['lat']])

In [ ]:
from sklearn.preprocessing import StandardScaler

# Cria uma instância da classe StandardScaler
scalerStandart = StandardScaler()

# Normaliza a coluna desejada
main_dataTmp['long_nrmlzd_Standart'] = scalerStandart.fit_transform(main_dataTmp[['long']])

In [ ]:
from sklearn.preprocessing import StandardScaler

# Cria uma instância da classe StandardScaler
scalerStandart = StandardScaler()

# Normaliza a coluna desejada
if(altitude):
  main_dataTmp['altitude_nrmlzd_Standart'] = scalerStandart.fit_transform(main_dataTmp[['altitude']])

In [ ]:
from sklearn.preprocessing import StandardScaler

# Cria uma instância da classe StandardScaler
scalerStandart = StandardScaler()

# Normaliza a coluna desejada
if(fungicides):
  for col in main_dataTmp.columns:
    if(col.find('DOSE') != -1):
      main_dataTmp[col + '_nrmlzd_Standart'] = scalerStandart.fit_transform(main_dataTmp[[col]])



In [ ]:
from sklearn.preprocessing import StandardScaler

cols_to_normalize = ['RH_mean_mes_11', 'RH_sd_mes_11', 'RH_max_mes_11', 'RH_min_mes_11', 
                     'RH_mean_mes_12', 'RH_sd_mes_12', 'RH_max_mes_12', 'RH_min_mes_12', 
                     'pr_mean_mes_11', 'pr_sd_mes_11', 'pr_max_mes_11', 'pr_min_mes_11', 'pr_sum_mes_11', 
                     'pr_mean_mes_12', 'pr_sd_mes_12', 'pr_max_mes_12', 'pr_min_mes_12', 'pr_sum_mes_12', 
                     'ETo_mean_mes_11', 'ETo_sd_mes_11', 'ETo_max_mes_11', 'ETo_min_mes_11', 'ETo_sum_mes_11', 
                     'ETo_mean_mes_12', 'ETo_sd_mes_12', 'ETo_max_mes_12', 'ETo_min_mes_12', 'ETo_sum_mes_12', 
                     'Rs_max_mes_11', 'Rs_min_mes_11', 'Rs_mean_mes_11', 'Rs_sd_mes_11', 
                     'Rs_max_mes_12', 'Rs_min_mes_12', 'Rs_mean_mes_12', 'Rs_sd_mes_12', 
                     'u2_min_mes_11', 'u2_mean_mes_11', 'u2_sd_mes_11', 'u2_max_mes_11', 
                     'u2_min_mes_12', 'u2_mean_mes_12', 'u2_sd_mes_12', 'u2_max_mes_12', 
                     'Tmax_sd_mes_11', 'Tmax_mean_mes_11', 'Tmax_max_mes_11', 'Tmax_min_mes_11', 
                     'Tmax_sd_mes_12', 'Tmax_mean_mes_12', 'Tmax_max_mes_12', 'Tmax_min_mes_12', 
                     'Tmin_sd_mes_11', 'Tmin_min_mes_11', 'Tmin_max_mes_11', 'Tmin_mean_mes_11', 
                     'Tmin_sd_mes_12', 'Tmin_min_mes_12', 'Tmin_max_mes_12', 'Tmin_mean_mes_12']

scaler = StandardScaler()
if(era5):
  main_dataTmp[cols_to_normalize] = scaler.fit_transform(main_dataTmp[cols_to_normalize])


In [ ]:
main_dataTmp

,study,year,location,state,rep,n_spray,brand_name,group,active_ingred,sev,...,CYPR_DOSE_nrmlzd_Standart,PICO_DOSE_nrmlzd_Standart,TRIF_DOSE_nrmlzd_Standart,PROT_DOSE_nrmlzd_Standart,TEBU_DOSE_nrmlzd_Standart,PYRA_DOSE_nrmlzd_Standart,FLUX_DOSE_nrmlzd_Standart,BENZ_DOSE_nrmlzd_Standart,EPOX_DOSE_nrmlzd_Standart,BIXA_DOSE_nrmlzd_Standart
0,252,2015,CAMPO VERDE,MT,1,3,Alto 100,DMI,cyproconazole,71.5,...,3.680655,-0.563623,-0.534281,-0.416638,-0.385554,-0.375146,-0.38364,-0.390897,-0.249017,-0.249017
1,252,2015,CAMPO VERDE,MT,2,3,Alto 100,DMI,cyproconazole,67.2,...,3.680655,-0.563623,-0.534281,-0.416638,-0.385554,-0.375146,-0.38364,-0.390897,-0.249017,-0.249017
2,252,2015,CAMPO VERDE,MT,3,3,Alto 100,DMI,cyproconazole,66.0,...,3.680655,-0.563623,-0.534281,-0.416638,-0.385554,-0.375146,-0.38364,-0.390897,-0.249017,-0.249017
3,252,2015,CAMPO VERDE,MT,4,3,Alto 100,DMI,cyproconazole,64.0,...,3.680655,-0.563623,-0.534281,-0.416638,-0.385554,-0.375146,-0.38364,-0.390897,-0.249017,-0.249017
4,269,2015,CAMPO VERDE,MT,1,3,Alto 100,DMI,cyproconazole,80.0,...,3.680655,-0.563623,-0.534281,-0.416638,-0.385554,-0.375146,-0.38364,-0.390897,-0.249017,-0.249017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12413,404,2019,EREBANGO,RS,4,4,Cronnos,QOI+DMI+DIT,picoxystrobin+tebuconazole+mancozeb,71.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12414,408,2019,SÃO MIGUEL DO PASSA QUATRO,GO,1,4,Cronnos,QOI+DMI+DIT,picoxystrobin+tebuconazole+mancozeb,28.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12415,408,2019,SÃO MIGUEL DO PASSA QUATRO,GO,2,4,Cronnos,QOI+DMI+DIT,picoxystrobin+tebuconazole+mancozeb,27.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12416,408,2019,SÃO MIGUEL DO PASSA QUATRO,GO,3,4,Cronnos,QOI+DMI+DIT,picoxystrobin+tebuconazole+mancozeb,26.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def remove_active_NaN(data=main_dataTmp):
    data.dropna(subset=['legenda_EN'],inplace=True)

if(solo):
  remove_active_NaN()

In [ ]:
def remove_active_NaN(data=main_dataTmp):
    data.dropna(subset=['AZOX_DOSE'],inplace=True)

if(fungicides):
  remove_active_NaN()

In [ ]:
main_dataTmp.columns

Index(['study', 'year', 'location', 'state', 'rep', 'n_spray', 'brand_name',
       'group', 'active_ingred', 'sev',
       ...
       'CYPR_DOSE_nrmlzd_Standart', 'PICO_DOSE_nrmlzd_Standart',
       'TRIF_DOSE_nrmlzd_Standart', 'PROT_DOSE_nrmlzd_Standart',
       'TEBU_DOSE_nrmlzd_Standart', 'PYRA_DOSE_nrmlzd_Standart',
       'FLUX_DOSE_nrmlzd_Standart', 'BENZ_DOSE_nrmlzd_Standart',
       'EPOX_DOSE_nrmlzd_Standart', 'BIXA_DOSE_nrmlzd_Standart'],
      dtype='object', length=151)

Remove not convenient columns

In [ ]:
main_dataTmp.drop(columns=['brand_name_EN', 'RH_mean_mes_11', 'RH_sd_mes_11', 'RH_max_mes_11',
       'RH_min_mes_11', 'RH_mean_mes_12', 'RH_sd_mes_12', 'RH_max_mes_12',
       'RH_min_mes_12', 'pr_mean_mes_11', 'pr_sd_mes_11', 'pr_max_mes_11',
       'pr_min_mes_11', 'pr_sum_mes_11', 'pr_mean_mes_12', 'pr_sd_mes_12',
       'pr_max_mes_12', 'pr_min_mes_12', 'pr_sum_mes_12', 'ETo_mean_mes_11',
       'ETo_sd_mes_11', 'ETo_max_mes_11', 'ETo_min_mes_11', 'ETo_sum_mes_11',
       'ETo_mean_mes_12', 'ETo_sd_mes_12', 'ETo_max_mes_12', 'ETo_min_mes_12',
       'ETo_sum_mes_12', 'Rs_max_mes_11', 'Rs_min_mes_11', 'Rs_mean_mes_11',
       'Rs_sd_mes_11', 'Rs_max_mes_12', 'Rs_min_mes_12', 'Rs_mean_mes_12',
       'Rs_sd_mes_12', 'u2_min_mes_11', 'u2_mean_mes_11', 'u2_sd_mes_11',
       'u2_max_mes_11', 'u2_min_mes_12', 'u2_mean_mes_12', 'u2_sd_mes_12',
       'u2_max_mes_12', 'Tmax_sd_mes_11', 'Tmax_mean_mes_11',
       'Tmax_max_mes_11', 'Tmax_min_mes_11', 'Tmax_sd_mes_12',
       'Tmax_mean_mes_12', 'Tmax_max_mes_12', 'Tmax_min_mes_12',
       'Tmin_sd_mes_11', 'Tmin_min_mes_11', 'Tmin_max_mes_11',
       'Tmin_mean_mes_11', 'Tmin_sd_mes_12', 'Tmin_min_mes_12',
       'Tmin_max_mes_12', 'Tmin_mean_mes_12'], inplace=True)

<ipython-input-48-dd4368653a7f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_dataTmp.drop(columns=['brand_name_EN', 'RH_mean_mes_11', 'RH_sd_mes_11', 'RH_max_mes_11',


In [ ]:


try:
  main_dataTmp.drop(columns=["group", "active_ingred", "yld", 'study', 'lat', 'long'], inplace=True)
except:
  print("ERRO 1")

try:
  main_dataTmp.drop(columns=['altitude'], inplace=True)
except:
  print("ERRO 2")

try:
  main_dataTmp.drop(columns=['desc_completa', 'distribuicao_umidade', 'tipo_umidade'], inplace=True)
except:
   print("ERRO 3")

try:
  main_dataTmp.drop(columns=['legenda', 'ordem', 'subordem', 'grande_grupo', 'subgrupos', 'textura', 'relevo'], inplace=True)
except:
  print("ERRO 4")

try:
   main_dataTmp.drop(columns=['AZOX_DOSE', 'CYPR_DOSE',
       'PICO_DOSE', 'TRIF_DOSE', 'PROT_DOSE', 'TEBU_DOSE', 'PYRA_DOSE',
       'FLUX_DOSE', 'BENZ_DOSE', 'EPOX_DOSE', 'BIXA_DOSE'], inplace=True)
   main_dataTmp.drop(columns=['Unnamed: 0', 'Code', 'Dose g a.i./ha', 'Comercial name', 'L or kg c.p./ha', 'n'], inplace=True)
except:
  print("ERRO 5")


main_dataTmp

,year,location,state,rep,n_spray,brand_name,sev,brand_name_EN,RH_mean_mes_11,RH_sd_mes_11,...,CYPR_DOSE_nrmlzd_Standart,PICO_DOSE_nrmlzd_Standart,TRIF_DOSE_nrmlzd_Standart,PROT_DOSE_nrmlzd_Standart,TEBU_DOSE_nrmlzd_Standart,PYRA_DOSE_nrmlzd_Standart,FLUX_DOSE_nrmlzd_Standart,BENZ_DOSE_nrmlzd_Standart,EPOX_DOSE_nrmlzd_Standart,BIXA_DOSE_nrmlzd_Standart
0,2015,CAMPO VERDE,MT,1,3,Alto 100,71.5,0,-0.458631,1.500566,...,3.680655,-0.563623,-0.534281,-0.416638,-0.385554,-0.375146,-0.38364,-0.390897,-0.249017,-0.249017
1,2015,CAMPO VERDE,MT,2,3,Alto 100,67.2,0,-0.458631,1.500566,...,3.680655,-0.563623,-0.534281,-0.416638,-0.385554,-0.375146,-0.38364,-0.390897,-0.249017,-0.249017
2,2015,CAMPO VERDE,MT,3,3,Alto 100,66.0,0,-0.458631,1.500566,...,3.680655,-0.563623,-0.534281,-0.416638,-0.385554,-0.375146,-0.38364,-0.390897,-0.249017,-0.249017
3,2015,CAMPO VERDE,MT,4,3,Alto 100,64.0,0,-0.458631,1.500566,...,3.680655,-0.563623,-0.534281,-0.416638,-0.385554,-0.375146,-0.38364,-0.390897,-0.249017,-0.249017
4,2015,CAMPO VERDE,MT,1,3,Alto 100,80.0,0,-0.458631,1.500566,...,3.680655,-0.563623,-0.534281,-0.416638,-0.385554,-0.375146,-0.38364,-0.390897,-0.249017,-0.249017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11024,2019,EREBANGO,RS,4,4,Vessarya,71.5,25,-0.458817,-1.003793,...,-0.491508,1.774235,-0.534281,-0.416638,-0.385554,-0.375146,-0.38364,2.558218,-0.249017,-0.249017
11025,2019,SÃO MIGUEL DO PASSA QUATRO,GO,1,4,Vessarya,25.8,25,-0.458908,-0.752673,...,-0.491508,1.774235,-0.534281,-0.416638,-0.385554,-0.375146,-0.38364,2.558218,-0.249017,-0.249017
11026,2019,SÃO MIGUEL DO PASSA QUATRO,GO,2,4,Vessarya,24.8,25,-0.458908,-0.752673,...,-0.491508,1.774235,-0.534281,-0.416638,-0.385554,-0.375146,-0.38364,2.558218,-0.249017,-0.249017
11027,2019,SÃO MIGUEL DO PASSA QUATRO,GO,3,4,Vessarya,25.8,25,-0.458908,-0.752673,...,-0.491508,1.774235,-0.534281,-0.416638,-0.385554,-0.375146,-0.38364,2.558218,-0.249017,-0.249017


In [ ]:
main_dataTmp

,year,location,state,rep,n_spray,brand_name,sev,tipo_umidade_EN,distribuicao_umidade_EN,desc_completa_EN_P1,...,CYPR_DOSE_nrmlzd_Standart,PICO_DOSE_nrmlzd_Standart,TRIF_DOSE_nrmlzd_Standart,PROT_DOSE_nrmlzd_Standart,TEBU_DOSE_nrmlzd_Standart,PYRA_DOSE_nrmlzd_Standart,FLUX_DOSE_nrmlzd_Standart,BENZ_DOSE_nrmlzd_Standart,EPOX_DOSE_nrmlzd_Standart,BIXA_DOSE_nrmlzd_Standart
0,2015,CAMPO VERDE,MT,1,3,Alto 100,71.5,0,4,0,...,3.680655,-0.563623,-0.534281,-0.416638,-0.385554,-0.375146,-0.38364,-0.390897,-0.249017,-0.249017
1,2015,CAMPO VERDE,MT,2,3,Alto 100,67.2,0,4,0,...,3.680655,-0.563623,-0.534281,-0.416638,-0.385554,-0.375146,-0.38364,-0.390897,-0.249017,-0.249017
2,2015,CAMPO VERDE,MT,3,3,Alto 100,66.0,0,4,0,...,3.680655,-0.563623,-0.534281,-0.416638,-0.385554,-0.375146,-0.38364,-0.390897,-0.249017,-0.249017
3,2015,CAMPO VERDE,MT,4,3,Alto 100,64.0,0,4,0,...,3.680655,-0.563623,-0.534281,-0.416638,-0.385554,-0.375146,-0.38364,-0.390897,-0.249017,-0.249017
4,2015,CAMPO VERDE,MT,1,3,Alto 100,80.0,0,4,0,...,3.680655,-0.563623,-0.534281,-0.416638,-0.385554,-0.375146,-0.38364,-0.390897,-0.249017,-0.249017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11024,2019,EREBANGO,RS,4,4,Vessarya,71.5,2,0,2,...,-0.491508,1.774235,-0.534281,-0.416638,-0.385554,-0.375146,-0.38364,2.558218,-0.249017,-0.249017
11025,2019,SÃO MIGUEL DO PASSA QUATRO,GO,1,4,Vessarya,25.8,0,4,0,...,-0.491508,1.774235,-0.534281,-0.416638,-0.385554,-0.375146,-0.38364,2.558218,-0.249017,-0.249017
11026,2019,SÃO MIGUEL DO PASSA QUATRO,GO,2,4,Vessarya,24.8,0,4,0,...,-0.491508,1.774235,-0.534281,-0.416638,-0.385554,-0.375146,-0.38364,2.558218,-0.249017,-0.249017
11027,2019,SÃO MIGUEL DO PASSA QUATRO,GO,3,4,Vessarya,25.8,0,4,0,...,-0.491508,1.774235,-0.534281,-0.416638,-0.385554,-0.375146,-0.38364,2.558218,-0.249017,-0.249017


In [ ]:
#main_dataTmp.to_excel("analise_graficos_dados_modificados_sem_solo.xlsx")

Studying data

In [ ]:
pd.set_option('display.max_rows', 120)
main_dataTmp.dtypes

year                              int64
location                         object
state                            object
rep                               int64
n_spray                           int64
brand_name                       object
sev                             float64
tipo_umidade_EN                   int64
distribuicao_umidade_EN           int64
desc_completa_EN_P1               int64
desc_completa_EN_P2               int64
OH_desc_completa_Tropical         int64
OH_desc_completa_Equatorial       int64
OH_desc_completa_Temperado        int64
OH_desc_completa_Quente           int64
OH_desc_completa_Subquente        int64
OH_desc_completa_Mesotermico      int64
OH_AZOX                         float64
OH_CYPR                         float64
OH_PICO                         float64
OH_TRIF                         float64
OH_PROT                         float64
OH_TEBU                         float64
OH_PYRA                         float64
OH_FLUX                         float64


In [ ]:
pd.set_option('display.max_rows', 120)
corr_matrix = main_dataTmp.corr()
corr_matrix["sev"].sort_values(ascending=True)

<ipython-input-47-5a054fbdcf9b>:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = main_dataTmp.corr()


QOI                            -0.495292
SDHI                           -0.366028
OH_PROT                        -0.243201
PROT_DOSE_nrmlzd_Standart      -0.243194
OH_BENZ                        -0.237381
BENZ_DOSE_nrmlzd_Standart      -0.237381
OH_TRIF                        -0.228000
TRIF_DOSE_nrmlzd_Standart      -0.220478
OH_BIXA                        -0.157645
BIXA_DOSE_nrmlzd_Standart      -0.157645
OH_FLUX                        -0.156673
OH_PYRA                        -0.156673
OH_PICO                        -0.152084
PICO_DOSE_nrmlzd_Standart      -0.152084
FLUX_DOSE_nrmlzd_Standart      -0.147035
OH_desc_completa_Subquente     -0.145384
PYRA_DOSE_nrmlzd_Standart      -0.140374
EPOX_DOSE_nrmlzd_Standart      -0.128323
OH_EPOX                        -0.128323
DMI                            -0.125552
year                           -0.119227
desc_completa_EN_P2            -0.116736
tipo_umidade_EN                -0.113940
OH_desc_completa_Tropical      -0.083191
subordem_EN     

In [ ]:
from pandas.plotting import scatter_matrix

if(plotar):
  fig = scatter_matrix(main_dataTmp, figsize=(12, 8))
  for ax in fig.ravel():
    ax.set_xlabel(ax.get_xlabel(), fontsize = 20, rotation = 90)
    ax.set_ylabel(ax.get_ylabel(), rotation = 0, labelpad=100,)

In [ ]:
main_dataTmp.drop(columns=['year', 'location', 'state', 'brand_name'], inplace=True)

In [ ]:
#Exportar os dados integrados
#main_dataTmp.to_excel("dadosLimpoParaComparar.xlsx")

Splitting data and training

In [ ]:
def split_train_test(data=main_dataTmp, test_ratio=.2):
    np.random.seed(42)
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]
train_set, test_set = split_train_test()

In [ ]:
#import sklearn
from sklearn.tree import ExtraTreeRegressor
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn import svm

from sklearn.metrics import mean_squared_error

#lin_reg = linear_model.LinearRegression()
#lin_reg = DecisionTreeRegressor()
lin_reg = RandomForestRegressor()
#lin_reg = KernelRidge(alpha=1.0)
#lin_reg = linear_model.Ridge(alpha=.5)
#lin_reg = linear_model.BayesianRidge()
#lin_reg = ExtraTreeRegressor()

lin_reg.fit(train_set.drop("sev", axis=1), train_set["sev"])


sev_predictions1 = lin_reg.predict(train_set.drop("sev", axis=1))
lin_mse1 = mean_squared_error(train_set["sev"], sev_predictions1)
lin_rmse1 = np.sqrt(lin_mse1)
sev_predictions2 = lin_reg.predict(test_set.drop("sev", axis=1))
lin_mse2 = mean_squared_error(test_set["sev"], sev_predictions2)
lin_rmse2 = np.sqrt(lin_mse2)

print("Erro no set de treino:", lin_rmse1, ", Erro no set de teste: ", lin_rmse2)














#COM ALTITUDE
#Extra tree: Erro no set de treino: 8.073521001098314 , Erro no set de teste:  11.61327043637036
#Basyesian Ridge: Erro no set de treino: 19.64516360328362 , Erro no set de teste:  18.96937107153677
#Ridge: Erro no set de treino: 19.646652962833457 , Erro no set de teste:  18.971945889252694
#Kernel ridge: Erro no set de treino: 19.65770707551919 , Erro no set de teste:  18.974898520473822
#Random Forest: Erro no set de treino: 8.225527478556199 , Erro no set de teste:  10.58194985313263
#DecisionTree: Erro no set de treino: 8.07350281821003 , Erro no set de teste:  11.548987719625721
#Linear: Erro no set de treino: 19.64383976835559 , Erro no set de teste:  18.9655915690511


#SEM ALTITUDE
#Extra tree: Erro no set de treino: 8.181730321963844 , Erro no set de teste:  12.629762697676199
#Basyesian Ridge: Erro no set de treino: 20.085694948746585 , Erro no set de teste:  19.417896051763556
#Ridge: Erro no set de treino: 19.646652962833457 , Erro no set de teste:  18.971945889252694
#Kernel ridge: Erro no set de treino: 20.084757954607994 , Erro no set de teste:  19.422465530549015
#Random Forest: Erro no set de treino: 8.32118188964604 , Erro no set de teste:  11.507957431180674
#DecisionTree: Erro no set de treino: 8.181523385284308 , Erro no set de teste:  12.352884150653262
#Linear: Erro no set de treino: 20.084715018624443 , Erro no set de teste:  19.423896583733715

Erro no set de treino: 6.734208238844401 , Erro no set de teste:  12.082036189370259


In [ ]:
import os
import graphviz
from sklearn.tree import DecisionTreeRegressor
from sklearn.datasets import make_regression
from sklearn.tree import export_graphviz

output_folder = "./output_folder"
os.makedirs(output_folder, exist_ok=True)

output_path = os.path.join(output_folder, "decision_tree.pdf")
dot_data = export_graphviz(lin_reg, out_file=None, feature_names=[f"feature_{i}" for i in range(train_set.drop("sev", axis=1).shape[1])])
graph = graphviz.Source(dot_data)
graph.format = "pdf"
graph.render(output_path)

print("Decision Tree exported as a PDF file successfully.")

Decision Tree exported as a PDF file successfully.


In [ ]:
main_dataTmp.columns.values


In [ ]:
output_file = open("resultados.csv", "w")
output_file.write("Dados, opções, modelo, erro\n")
options = ""
dadosSelecionados = selectedData()
print(dadosSelecionados)

altitude+climaIBGE+solo+classeFungicida+fungicides+era5


In [ ]:
def split_train_test(data=main_dataTmp, test_ratio=.2):
    np.random.seed(42)
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]
train_set, test_set = split_train_test()

In [ ]:
#import sklearn

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

lin_reg = RandomForestRegressor()

lin_reg.fit(train_set.drop("sev", axis=1), train_set["sev"])


sev_predictions1 = lin_reg.predict(train_set.drop("sev", axis=1))
lin_mse1 = mean_squared_error(train_set["sev"], sev_predictions1)
lin_rmse1 = np.sqrt(lin_mse1)
sev_predictions2 = lin_reg.predict(test_set.drop("sev", axis=1))
lin_mse2 = mean_squared_error(test_set["sev"], sev_predictions2)
lin_rmse2 = np.sqrt(lin_mse2)

output_file = open("resultados.csv", "a")
options = ""

output_file.write(dadosSelecionados + "," + "" + "," + "Random Forest" + "," + str(lin_rmse2) + "\n")
output_file.close()
print("Erro no set de treino:", lin_rmse1, ", Erro no set de teste: ", lin_rmse2)


Erro no set de treino: 6.739502827821489 , Erro no set de teste:  11.928488615425936


In [ ]:
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error


lin_reg = CatBoostRegressor(verbose=False)
#lin_reg = XGBRegressor()


lin_reg.fit(train_set.drop("sev", axis=1), train_set["sev"])


sev_predictions1 = lin_reg.predict(train_set.drop("sev", axis=1))
lin_mse1 = mean_squared_error(train_set["sev"], sev_predictions1)
lin_rmse1 = np.sqrt(lin_mse1)
sev_predictions2 = lin_reg.predict(test_set.drop("sev", axis=1))
lin_mse2 = mean_squared_error(test_set["sev"], sev_predictions2)
lin_rmse2 = np.sqrt(lin_mse2)

output_file = open("resultados.csv", "a")
options = ""
output_file.write(dadosSelecionados + "," + "" + "," + "CatBoost" + "," + str(lin_rmse2) + "\n")
output_file.close()
print("Erro no set de treino:", lin_rmse1, ", Erro no set de teste: ", lin_rmse2)

Erro no set de treino: 7.370918524778902 , Erro no set de teste:  10.449951421639234


In [ ]:
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error


lin_reg = CatBoostRegressor(verbose=False, iterations=100)
#lin_reg = XGBRegressor()


lin_reg.fit(train_set.drop("sev", axis=1), train_set["sev"])


sev_predictions1 = lin_reg.predict(train_set.drop("sev", axis=1))
lin_mse1 = mean_squared_error(train_set["sev"], sev_predictions1)
lin_rmse1 = np.sqrt(lin_mse1)
sev_predictions2 = lin_reg.predict(test_set.drop("sev", axis=1))
lin_mse2 = mean_squared_error(test_set["sev"], sev_predictions2)
lin_rmse2 = np.sqrt(lin_mse2)

output_file = open("resultados.csv", "a")
options = ""
output_file.write(dadosSelecionados + "," + "Iterations=100" + "," + "CatBoost" + "," + str(lin_rmse2) + "\n")
output_file.close()
print("Erro no set de treino:", lin_rmse1, ", Erro no set de teste: ", lin_rmse2)

Erro no set de treino: 7.824912599606304 , Erro no set de teste:  9.881266700309656


In [ ]:
import sklearn
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

lin_reg = MLPRegressor(hidden_layer_sizes=(80,40), activation='relu', solver='adam', max_iter=4000)

# utilizando cross validation com 5 folds


lin_reg.fit(train_set.drop("sev", axis=1), train_set["sev"])

sev_predictions1 = lin_reg.predict(train_set.drop("sev", axis=1))
lin_mse1 = mean_squared_error(train_set["sev"], sev_predictions1)
lin_rmse1 = np.sqrt(lin_mse1)
sev_predictions2 = lin_reg.predict(test_set.drop("sev", axis=1))
lin_mse2 = mean_squared_error(test_set["sev"], sev_predictions2)
lin_rmse2 = np.sqrt(lin_mse2)

output_file = open("resultados.csv", "a")
options = ""
output_file.write(dadosSelecionados + "," + "" + "," + "Rede Neural" + "," + str(lin_rmse2) + "\n")
output_file.close()
print("Erro no set de treino:", lin_rmse1, ", Erro no set de teste: ", lin_rmse2)

Erro no set de treino: 7.774448442662666 , Erro no set de teste:  10.045717230019543
